# 05 Display An Interactive Map
---

Using the [`folium`](https://github.com/python-visualization/folium) package to display a basic interative map using the OS Data Hub OS Maps API.

### Folium
---

The [folium](https://python-visualization.github.io/folium/) Python package provides a wrapper around the ubiquitous open-source JavaScript web mapping library [Leaflet.js](https://leafletjs.com/). It allows the visualisation of geospatial data in Python on an interactive leaflet map.

### OS Data Hub
---

The OS Data Hub is the new data portal to access Ordnance Survey (OS) data. Data can be downloaded through a user interace or programatically via a series of APIs enabling direct integration with software applications.

The OS Data Hub [Public Sector Plan](https://www.ordnancesurvey.co.uk/business-government/public-sector-geospatial-agreement/data-hub-for-public-sector) provides [PSGA members](https://www.ordnancesurvey.co.uk/business-government/sectors/public-sector) with unlimited access to OS OpenData and OS Premium data available under the PSGA via API or download.

### OS Maps API
---

The [OS Maps API](https://osdatahub.os.uk/docs/wmts/overview) is a mapping API available through the OS Data Hub. It allows OS base mapping to be embedded into mapping applications including a [tiled web map](https://en.wikipedia.org/wiki/Tiled_web_map) or slippy map in Python. The API serves pre-rendered raster tiles in two coordinate reference systems (CRS), British National Grid ([EPSG:27700](https://epsg.io/27700)) and Web Mercator ([EPSG:3857](https://epsg.io/3857)) which geospatial data can be overlayed ontop of. The raster tiles are also available in a range of styles to support different geodata visualisation use cases.

Use of the OS Maps API for both OpenData and Premium data map views is free to the Public Sector but requests still require an [API key](https://en.wikipedia.org/wiki/Application_programming_interface_key) for authenitcation. If you do not have access to the OS Data Hub under the Public Sector plan then you can still use the OS Maps API for free via the [OS OpenData plan](https://osdatahub.os.uk/plans).

### ZXY vs WMTS
---

The OS Maps API offers two different resources, ZXY and Web Map Tile Service (WMTS). Both resources are supported by folium but the code required to integrate a ZXY resource is less verbose and prefered.

With the ZXY resource the Z, X and Y provide the indices of a pre-rendered mapping tile within a request. Leaflet translates the extent of the map for a given zoom level (Z) into a series of API requests that will download the tiles required to fill the extent.

---

In [1]:
import geopandas as gpd
from datetime import datetime
from folium import LayerControl, Map, TileLayer
from folium.plugins import FloatImage

### OS Maps API ZXY resource

In [2]:
# OS Maps API layer name
# Example uses Light Style in Web Mercator (EPSG:3857) projection
layer = "Light_3857"
# OS Data Hub project API key
key = "frKhvBUiMB5DGwl3pGb2GzcOz6ApgyP0"

# OS Data Hub base path - https://api.os.uk
# OS Maps API ZXY end point path - /maps/raster/v1/zxy/
url = f"https://api.os.uk/maps/raster/v1/zxy/{layer}/{{z}}/{{x}}/{{y}}.png?key={key}"

### Create custom folium map

In [3]:
# OS logo image
logo_url = "https://raw.githubusercontent.com/OrdnanceSurvey/os-api-branding/master/img/os-logo-maps.svg"

# Folium FloatImage plugin for displaying image on Map
image = FloatImage(logo_url, bottom=1, left=1)

In [5]:
# Create Folium map
m = Map(
    location=[
        51.507,
        -0.105,
    ],  # Map centre coordinates (by convention latitude (y), longitude (x))
    tiles=url,
    attr=f"Contains OS data &copy; Crown copyright and database rights {datetime.now().year}",  # OS Data Hub attribution statement
    min_zoom=7,  # See EPSG:3857 Tile Matrix Set - https://osdatahub.os.uk/docs/wmts/technicalSpecification
    max_zoom=20,
    zoom_start=12,
)

# Add image to Map
image.add_to(m)

# Return Map
m

In [7]:
# Iterate over OS Maps layer styles
for layer in ["Light_3857", "Outdoor_3857", "Road_3857"]:
    url = (
        f"https://api.os.uk/maps/raster/v1/zxy/{layer}/{{z}}/{{x}}/{{y}}.png?key={key}"
    )
    
    # Create tile layer
    TileLayer(
        location=[
            51.507,
            -0.105,
        ],  # Map centre coordinates (by convention latitude (y), longitude (x))
        tiles=url,
        attr=f"Contains OS data &copy; Crown copyright and database rights {datetime.now().year}",  # OS Data Hub attribution statement
        min_zoom=7,  # See EPSG:3857 Tile Matrix Set - https://osdatahub.os.uk/docs/wmts/technicalSpecification
        max_zoom=20,
        zoom_start=12,
        name=layer,
    ).add_to(m)

# Add layer 'switcher' to map
LayerControl().add_to(m)

m